#### Getting set up

- Move this notebook into dedicated folder - either within a 'Repos' folder if you already have one, or create one on your desktop
    - This will the folder to define as your 'workspace'
- To get started, you'll need to set up a virtual environment
- Click 'Select Kernel' in the top right corner, or press Ctrl+Shift+P and select/search 'Notebook: Select Notebok Kernel' from the menu
- Select 'Python Environments' -> 'Create Python Environment' -> 'Venv' -> Then choose a python interpreter (which you hopefully have installed from out last onsite)
    - If you don't have python installed, you should be able to get this yourself from Software Centre on your JLR laptop
- Uncomment the cell below and execute to install the required packages to connect to Google BigQuery


In [ ]:
%pip install pandas google-cloud-bigquery google-cloud-bigquery-storage db-dtypes

In [ ]:
# TODO: You'll need to install the google cloud SDK from software centre and sign in via the SDK interface; run 'gcloud init'
# TODO: Once signed in, run 'gcloud auth application-default login' from SDK interface
# TODO: When ready, install the required packages from the cell above (uncomment first!)

from google.cloud import bigquery
import pandas as pd
import os

# TODO: Write a query to select the data from the 'sellers_corrupted' table within the 'analytics_onsite' dataset from the 'jlr-dl-cat-training' environment
query = """
SELECT *
FROM `jlr-dl-cat-training.analytics_onsite.sellers_corrupted` 
"""

# Initialize BigQuery client
client = bigquery.Client(project="jlr-dl-cat-training") # TODO:Enter the project which will be billed here

# Run the query and load into a DataFrame
df = client.query(query).to_dataframe()

2. Display basic info: shape, column names, data types. Preview the first 10 row of data.


In [ ]:
#TODO: Report on the shape of the data, column names, data types.
df.shape
df.info()

#TODO: Preview the first 10 row of data.
df.head(10)

#### Exercise 1: Standardizing Text Case

**Goal:** Normalize text:
- `seller_city` → lowercase and whitespaces removed
- `seller_state` → uppercase and whitespaces removed
- Assign to new columns ('seller_city_clean', 'seller_state_clean') to preserve original data

**Hint:**
- String functions can be applied using `.str.____`
- Functions can be strung together


In [ ]:
# Standardize city and state names
#TODO: Write code here, reference new columns 'seller_city_clean', 'seller_state_clean'
df['seller_city_clean'] = df['seller_city'].str.lower().str.replace(' ', '')
df['seller_state_clean'] = df['seller_state'].str.upper().str.replace(' ', '')

# Preview changes
filtered_df = df[df['seller_city'].str.contains(r'[A-Z]', na=False)]
filtered_df[['seller_city', 'seller_city_clean', 'seller_state', 'seller_state_clean']].head()

#### Exercise 2: Identifying Duplicates

**Goal:** Count duplicate data entries, then remove them

**Hint:**
- Use `.sum()` to count

In [ ]:
#TODO: Count duplicate entries
df.duplicated().sum()

#TODO: Remove duplicate entries
df1 = df.drop_duplicates()
df1.duplicated().sum()

#TODO: Verify any duplicate entries have been removed
df.info()
df1.info()


### Exercise 3: Counting Unique Values per Column

**Goal:** Determine how many unique values exist in each column of the dataset.

**Instructions:**
- Use the `.nunique()` method to count unique values.
- Display the result of each column in a summary table

In [ ]:
#TODO: Count unique entries for each column
df1.nunique()


#### Exercise 4: Detecting Whitespace Issues

**Goal:** Identify rows in `seller_city` and `seller_state` with:
- Leading/trailing whitespace (these were removed in exercise 1, but this time we want to detect them first, not just blindly removed them)
- Multiple consecutive spaces
- Cells that are only whitespace

**Hint:**
- You will need to evaluate whether the value meets certain conditions
- Use `'  '` (double space) to check for multiple consecutive spaces
- Check whether `.str.strip()` also results in a blank cell

In [ ]:
# Detect whitespace issues in string columns
def detect_whitespace_issues(series):
    return series.apply(lambda x: isinstance(x, str) and (
        #TODO: insert logic here to identify problematic values
        x != x.strip() or '  ' in x or x.strip() == ""
        )        )

# Apply to relevant columns
for col in ['seller_city', 'seller_state']:
    issues = detect_whitespace_issues(df[col])
    if issues.any():
        print(f"Whitespace issues in column '{col}':")
        display(df.loc[issues, [col]])

#### Exploring dataframes in python

It's often difficult to understand how dataframes look using pure python, but a little easier using Notebooks.
<br> However, there are tools in VS Code to help you interrogate data tables!

**Goal:** Install the 'Data Wrangler' Extension and explore the dataframe; sort, filter or clean data.
- Use the UI to understand how to turn your interactions into python code

**Hint:**
- Data wrangler is not a python package, it's a VS Code extension!

In [ ]:
#TODO: Install the 'Data Wrangler' Extension. You don't need to write any code here.
print(df)

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    # Convert text to uppercase in column: 'seller_city'
    df['seller_city'] = df['seller_city'].str.upper()
    # Convert text to lowercase in column: 'seller_state'
    df['seller_state'] = df['seller_state'].str.lower()
    return df

df_clean = clean_data(df.copy())
df_clean.head()

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df):
    # Drop duplicate rows across all columns
    df = df.drop_duplicates()
    return df

df_clean = clean_data(df.copy())
df_clean.head()

#### Automating data exploration

- Understanding data before diving into analysis is often skipped, a crucial step when working with a dataset for the first time.
    - Without a clear grasp of the structure, quality, and context of your data, you risk drawing misleading conclusions or missing key insights altogether.
<br> 
- This isn’t a one-time skill - it’s a recurring discipline. Every new dataset brings its own quirks, inconsistencies, and hidden patterns.
    - Whether it’s missing values, unexpected formats, or outliers, each dataset demands a fresh round of exploration and validation.
<br> 
- Automating parts of this initial exploration can dramatically speed up the process and reduce errors from misunderstandings.
    - It frees up time for deeper, more strategic thinking and ensures a consistent, repeatable approach to data understanding across projects.
<br> 

**Goal:** Use `pandas-dq` and `ydata-profiling` to understand the dataframe
- Install the `pandas-dq` and `ydata-profiling`
- Run the cells below

**Hint:**
- You can install packages directly from the 
- Change the verbose setting between 0 and 1 for `pandas-dq`
- `ydata-profiling` you will likely encounter errors relating to package dependencies and versions, resolve the dependencies to install successfully

In [ ]:
# TODO: install pandas-dq

# %pip install pandas-dq
# TODO: install ydata-profiling
%pip install ydata_profiling


In [ ]:
import pandas_dq as dq

dq_report = dq.dq_report(df1, target= None, verbose=1)


In [ ]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Data Quality Report", explorative=True)
profile.to_notebook_iframe()

#### Advanced challenge:

Highly cardinal columns (those with a large number of unique values) can pose significant challenges in data analysis and modelling.
There are several non-modeling reasons to resolve high-cardinality columns into groups:
- **Clarity**: Charts and summaries become cluttered and unreadable with too many categories
- **Insight Discovery**: Grouping helps highlight broader trends and patterns that might be obscured by granular detail
- **Comparability**: It’s easier to compare groups than hundreds of individual categories
- **Audience Understanding**: Humans struggle to draw conclusions from highly fragmented data due to cognitive overload
- **Storytelling**: Grouped data supports clearer narratives and more compelling insights
- **Error Detection**: High cardinality can mask typos, inconsistent naming, or duplicate entries (e.g., "NYC", "New York", "NewYork")
- **Standardization**: Grouping helps enforce consistency across datasets, especially when merging or joining data from multiple sources



**Task:**
The text is the 'seller_city' column has high cardinality (lots of unique values), due a mixture of formatting, spelling errors and different naming conventions created by different users.

Develop a method to cluster/categorise/reassign the seller city into a smaller number of cities.

**Hint:**
- Use information from other columns to aid your reclassification
- You're not expected to know how to do this from memory! Use co-pilot to translate your strategy into code.

For examples, run the cell below:

In [ ]:
%pip install rapidfuzz

In [ ]:

import pandas as pd
import numpy as np
from rapidfuzz import fuzz #TODO: Run the cell above to install rapidfuzz first

# Convert column to list of strings
values = df["seller_city"].astype(str).tolist()
distinct_values = list(dict.fromkeys(values))
n = len(distinct_values)

# Initialize similarity matrix
similarity_matrix = np.zeros((n, n))

# Compute pairwise similarity scores
for i in range(n):
    for j in range(n):
        similarity_matrix[i, j] = fuzz.ratio(distinct_values[i], distinct_values[j])

# Create a DataFrame for the similarity matrix
sim_df = pd.DataFrame(similarity_matrix, index=distinct_values, columns=distinct_values)

# Extract top N most similar city name pairs (excluding self-comparisons)
top_n = 20
similarities = []

for i in range(n):
    for j in range(i + 1, n):
        similarities.append((distinct_values[i], distinct_values[j], similarity_matrix[i, j]))

# Sort by similarity score in descending order
similarities.sort(key=lambda x: x[2], reverse=True)

# Display top N most similar pairs
for city1, city2, score in similarities[:top_n]:
    print(f"{city1} <-> {city2}")
